## Trending Youtube Video Statistic: Data Cleaning and Column Derivation

### Derivation of DataFrame from raw datasets

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
#glob is used to read files in any similar format in the same folder path
csv_files = [x for x in glob.glob('../GA_Project4_Draft/*.{}'.format('csv'))]

videos = []
country_list = []
for csv in csv_files:
    original_videos = pd.read_csv(csv, index_col=None)
    original_videos['country'] = csv[21:23]
    country_list.append(csv[21:23])
    videos.append(original_videos) #add original_videos (dataframe) into videos (list)

all_videos = pd.concat(videos)

In [6]:
print(country_list)

['CA', 'DE', 'FR', 'GB', 'US']


In [7]:
all_videos1 = all_videos.drop(['thumbnail_link','description'],axis=1)

In [8]:
all_videos1['trending_date'] = pd.to_datetime(all_videos1['trending_date'],format='%y.%d.%m')

In [9]:
all_videos1['publish_time'] = pd.to_datetime(all_videos1['publish_time'],format='%Y-%m-%dT%H:%M:%S.%fZ')

In [10]:
all_videos1.insert(4,'publish_date',all_videos1['publish_time'].dt.date)
all_videos1['publish_time'] = all_videos1['publish_time'].dt.time

In [11]:
all_videos1['publish_date'] = pd.to_datetime(all_videos1['publish_date'])

In [12]:
all_videos1['tags'] = all_videos1['tags'].apply(lambda x: x.replace('"',''))

In [13]:
all_videos1['tags'] = all_videos1['tags'].apply(lambda x: x.replace('/','|'))

In [14]:
all_videos1['refined_tags'] = all_videos1.tags.map(lambda x: [i.strip() for i in x.split("|")])
#if 'tags' is not a list, need to parse it and convert it into a list
#strip will remove selected characters
#split will split out Eminen|walk to ['Eminen','walk']

In [15]:
all_videos1['number_of_tags'] = all_videos1.refined_tags.apply(len)

In [16]:
all_videos1['net_likes_dislikes'] = all_videos1['likes']
all_videos1['net_likes_dislikes'] = all_videos1[['net_likes_dislikes']].sub(all_videos1['dislikes'],axis=0)

In [17]:
all_videos1['publish_hour'] = all_videos1['publish_time'].astype(str).str[:2]

In [18]:
#calculate trend_publish_diff in days
all_videos1['trend_publish_diff'] = (all_videos1['trending_date'] - all_videos1['publish_date']).dt.days

In [19]:
# create a dictionary of dataframe that is classified by country
d = {}
for i in range(0,len(country_list)):
    d[country_list[i]] = all_videos1[all_videos1.country == country_list[i]]

In [20]:
j = {}    
for i in country_list:
    # (1) groupby video_id and select min trend_publish_diff
    idx = d[i].groupby(['video_id'])['trend_publish_diff'].transform(min) == d[i]['trend_publish_diff']
    # (2) get the filtered dataframe
    j[i] = d[i][idx] #grouped dataframe
    # (3) get the series value on trend count based on original dataframe
    trend_count = d[i].video_id.value_counts()
    # (4) convert series from (3) to dataframe
    count_df = pd.DataFrame({'id':trend_count.index,'trend_count':trend_count.values})
    # (5) merge (2) and (4)
    j[i] = pd.merge(j[i],count_df,how='inner',left_on='video_id',right_on='id')
    j[i].drop(['id'],axis=1,inplace=True)

In [21]:
videos_byregion = []
for i in country_list:
    all_videos3 = pd.DataFrame.from_dict(j[i])
    videos_byregion.append(all_videos3)

final_videos = pd.concat(videos_byregion)
final_videos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,comments_disabled,ratings_disabled,video_error_or_removed,country,refined_tags,number_of_tags,net_likes_dislikes,publish_hour,trend_publish_diff,trend_count
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,2017-11-10,10,17:00:03,Eminem|Walk|On|Water|Aftermath|Shady|Interscop...,17158579,787425,...,False,False,False,CA,"[Eminem, Walk, On, Water, Aftermath, Shady, In...",8,744005,17,4,4
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,2017-11-13,23,17:00:00,plush|bad unboxing|unboxing|fan mail|idubbbztv...,1014651,127794,...,False,False,False,CA,"[plush, bad unboxing, unboxing, fan mail, idub...",11,126106,17,1,5
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,racist superman|rudy|mancuso|king|bach|racist|...,3191434,146035,...,False,False,False,CA,"[racist superman, rudy, mancuso, king, bach, r...",23,140696,19,2,4
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,ryan|higa|higatv|nigahiga|i dare you|idy|rhpc|...,2095828,132239,...,False,False,False,CA,"[ryan, higa, higatv, nigahiga, i dare you, idy...",14,130250,18,2,4
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,2017-11-09,10,11:04:14,edsheeran|ed sheeran|acoustic|live|cover|offic...,33523622,1634130,...,False,False,False,CA,"[edsheeran, ed sheeran, acoustic, live, cover,...",10,1613048,11,5,2


In [22]:
final_videos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94327 entries, 0 to 6374
Data columns (total 22 columns):
video_id                  94327 non-null object
trending_date             94327 non-null datetime64[ns]
title                     94327 non-null object
channel_title             94327 non-null object
publish_date              94327 non-null datetime64[ns]
category_id               94327 non-null int64
publish_time              94327 non-null object
tags                      94327 non-null object
views                     94327 non-null int64
likes                     94327 non-null int64
dislikes                  94327 non-null int64
comment_count             94327 non-null int64
comments_disabled         94327 non-null bool
ratings_disabled          94327 non-null bool
video_error_or_removed    94327 non-null bool
country                   94327 non-null object
refined_tags              94327 non-null object
number_of_tags            94327 non-null int64
net_likes_dislikes      

In [23]:
# all_videos1 dataframe has all rows and columns with all data cleaned up
df1 = pd.DataFrame(all_videos1)
df1.to_csv('all_videos1.csv')

In [24]:
# final_videos is grouped by country with the row of video id that get trended in the shortest period of time from publish date
df2 = pd.DataFrame(final_videos)
df2.to_csv('final_videos.csv')